In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm
from datetime import date 
import holidays
import lightgbm as lgb


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [2]:
# note you must run weather
site_time_offsets_df = pd.read_pickle('../input/ashrae-energy-prediction/site_time_offsets_df.pickle')

In [3]:
# note you must run create-holiday-df notebook first to create the pickle
holiday_df = pd.read_pickle('../input/ashrae-energy-prediction/holiday_df.pickle')

In [4]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [5]:
class AlignTime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['offset'] = df.site_id.map(site_time_offsets_df)
        df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
        df['timestamp'] = df['timestamp_aligned']
        del df['timestamp_aligned'], df['offset']
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self
    

In [6]:
weather_pre_pipes = Pipeline(
    steps=[
        ('convertToDatetime', ConvertToDatetime()),
        ('alignTime', AlignTime()),
    ]
)

In [7]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
    'weather' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16}
}

file_loc = {}    
for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
    for name in ['building_metadata','weather_train','weather_test','train','test']:
        if path.exists(dir_path + name + '.csv'):
            file_loc[name]= dir_path + name + '.csv'

    building = pd.read_csv(file_loc['building_metadata'], dtype=file_dtype['building_metadata'])
    weather_train = weather_pre_pipes.transform(pd.read_csv(file_loc['weather_train'], dtype=file_dtype['weather']))
    weather_test = weather_pre_pipes.transform(pd.read_csv(file_loc['weather_test'], dtype=file_dtype['weather']))
    train = ConvertToDatetime().transform(pd.read_csv(file_loc['train'], dtype=file_dtype['train']))
    test = ConvertToDatetime().transform(pd.read_csv(file_loc['test'], dtype=file_dtype['test']))


In [8]:
# "As you can see above, this data looks weired until May 20. It is 
# reported in this discussion by @barnwellguy that All electricity
# meter is 0 until May 20 for site_id == 0. Let's remove these data 
# from training data."
# https://www.kaggle.com/kaushal2896/ashrae-eda-fe-lightgbm-1-13
class RmS0M0(TransformerMixin):
  
    def transform(self, df, **transform_params):
        return df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

    def fit(self, X, y=None, **fit_params):
        return self
    


In [9]:
class ImputeWeather(TransformerMixin):

    def __init__(self, method:str='linear', gap_limit:int=None, limit_direction:str='forward'):
        self._method = method
        self._gap_limit = gap_limit
        self._limit_direction = limit_direction
        
    def transform(self, weather_df, **transform_params):
        grouped_weather_df = weather_df.groupby('site_id').apply(lambda group: group.interpolate(method=self._method, limit=self._gap_limit, limit_direction=self._limit_direction))
        if 'cloud_coverage' in grouped_weather_df.columns:
            grouped_weather_df['cloud_coverage'] = grouped_weather_df['cloud_coverage'].round(decimals=0).clip(0,8)
        grouped_weather_df.reset_index(inplace=True)
        weather_df = grouped_weather_df.drop(['index','Unnamed: 0'], axis=1)
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
print(weather_train.head(20))
print(ImputeWeather().transform(weather_train.head(20)))

    Unnamed: 0  site_id           timestamp  air_temperature  cloud_coverage  \
0            0        0 2015-12-31 19:00:00        25.000000             6.0   
1            1        0 2015-12-31 20:00:00        24.406250             NaN   
2            2        0 2015-12-31 21:00:00        22.796875             2.0   
3            3        0 2015-12-31 22:00:00        21.093750             2.0   
4            4        0 2015-12-31 23:00:00        20.000000             2.0   
5            5        0 2016-01-01 00:00:00        19.406250             NaN   
6            6        0 2016-01-01 01:00:00        21.093750             6.0   
7            7        0 2016-01-01 02:00:00        21.093750             NaN   
8            8        0 2016-01-01 03:00:00        20.593750             NaN   
9            9        0 2016-01-01 04:00:00        21.093750             NaN   
10          10        0 2016-01-01 05:00:00        21.093750             NaN   
11          11        0 2016-01-01 06:00

In [10]:
class AddWeatherLags(TransformerMixin):
    
    def __init__(self, window):
        self._window = window
        
    def transform(self, weather_df, **transform_params):
        group_df = weather_df.groupby(['site_id'])
        cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
        rolled = group_df[cols].rolling(window=self._window, min_periods=0)
        lag_mean = rolled.mean().reset_index().astype(np.float16)
        lag_max = rolled.max().reset_index().astype(np.float16)
        lag_min = rolled.min().reset_index().astype(np.float16)
        lag_std = rolled.std().reset_index().astype(np.float16)
        for col in cols:
            weather_df[f'{col}_mean_lag{self._window}'] = lag_mean[col]
            weather_df[f'{col}_max_lag{self._window}'] = lag_max[col]
            weather_df[f'{col}_min_lag{self._window}'] = lag_min[col]
            weather_df[f'{col}_std_lag{self._window}'] = lag_std[col]
        del group_df, rolled
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
    
print(AddWeatherLags(72).transform(weather_train.head(20)))

    Unnamed: 0  site_id           timestamp  air_temperature  cloud_coverage  \
0            0        0 2015-12-31 19:00:00        25.000000             6.0   
1            1        0 2015-12-31 20:00:00        24.406250             NaN   
2            2        0 2015-12-31 21:00:00        22.796875             2.0   
3            3        0 2015-12-31 22:00:00        21.093750             2.0   
4            4        0 2015-12-31 23:00:00        20.000000             2.0   
5            5        0 2016-01-01 00:00:00        19.406250             NaN   
6            6        0 2016-01-01 01:00:00        21.093750             6.0   
7            7        0 2016-01-01 02:00:00        21.093750             NaN   
8            8        0 2016-01-01 03:00:00        20.593750             NaN   
9            9        0 2016-01-01 04:00:00        21.093750             NaN   
10          10        0 2016-01-01 05:00:00        21.093750             NaN   
11          11        0 2016-01-01 06:00

[20 rows x 38 columns]


In [11]:
class AddBuilding(TransformerMixin):

    def __init__(self, building_df):
        self._b_df = building_df
        
    def transform(self, df, **transform_params):
        return df.merge(_b_df, on='building_id', how='left')

    def fit(self, X, y=None, **fit_params):
        return self

In [12]:
    
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114483#latest-660771
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114874#latest-660970
class AddHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
        return df

    def fit(self, X, y=None, **fit_params):
        return self
# Test 
addHolidays = AddHolidays()
print(addHolidays.transform(train.head(2000).merge(building, on='building_id', how='left'))[['building_id','timestamp','holiday']])

      building_id  timestamp         holiday
0               0 2016-01-01  New Year's Day
1               1 2016-01-01  New Year's Day
2               2 2016-01-01  New Year's Day
3               3 2016-01-01  New Year's Day
4               4 2016-01-01  New Year's Day
5               5 2016-01-01  New Year's Day
6               6 2016-01-01  New Year's Day
7               7 2016-01-01  New Year's Day
8               8 2016-01-01  New Year's Day
9               9 2016-01-01  New Year's Day
10             10 2016-01-01  New Year's Day
11             11 2016-01-01  New Year's Day
12             12 2016-01-01  New Year's Day
13             13 2016-01-01  New Year's Day
14             14 2016-01-01  New Year's Day
15             15 2016-01-01  New Year's Day
16             16 2016-01-01  New Year's Day
17             17 2016-01-01  New Year's Day
18             18 2016-01-01  New Year's Day
19             19 2016-01-01  New Year's Day
20             20 2016-01-01  New Year's Day
21        

In [13]:
class RmHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
        df = df.drop(df[df['holiday'].notnull()].index)
        df = df.drop(['holiday'], axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

# Test you should see the new years removed
rmHolidays = RmHolidays()
print(rmHolidays.transform(train.head(100000).merge(building, on='building_id', how='left')))

       building_id  meter           timestamp  meter_reading  site_id  \
55121            0      0 2016-01-02 00:00:00       0.000000        0   
55122            1      0 2016-01-02 00:00:00       0.000000        0   
55123            2      0 2016-01-02 00:00:00       0.000000        0   
55124            3      0 2016-01-02 00:00:00       0.000000        0   
55125            4      0 2016-01-02 00:00:00       0.000000        0   
55126            5      0 2016-01-02 00:00:00       0.000000        0   
55127            6      0 2016-01-02 00:00:00       0.000000        0   
55128            7      0 2016-01-02 00:00:00       0.000000        0   
55129            8      0 2016-01-02 00:00:00       0.000000        0   
55130            9      0 2016-01-02 00:00:00       0.000000        0   
55131           10      0 2016-01-02 00:00:00       0.000000        0   
55132           11      0 2016-01-02 00:00:00       0.000000        0   
55133           12      0 2016-01-02 00:00:00      

[41839 rows x 9 columns]


In [14]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(building.head(20)['square_feet'])

0       7432.0
1       2720.0
2       5376.0
3      23685.0
4     116607.0
5       8000.0
6      27926.0
7     121074.0
8      60809.0
9      27000.0
10    370773.0
11     49073.0
12     37100.0
13     99380.0
14     86250.0
15     83957.0
16     54644.0
17     15250.0
18    111891.0
19     18717.0
Name: square_feet, dtype: float32


In [15]:
class SetCatTypes(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['primary_use']= df['primary_use'].astype('category')
        df['meter'] = df["meter"].astype('category')
        df['site_id'] = df["site_id"].astype('category')
        df['building_id'] = df['building_id'].astype('category')
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [16]:
class ImputeCloudCoverage(TransformerMixin):
        
    def transform(self, df, **transform_params):
        # set age of building to mediam of site_id
        # else if set ot overall median
        median = df['cloud_coverage'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['cloud_coverage'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = i_median
            else:
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = median
        df['cloud_coverage'] = np.uint8(df['cloud_coverage'])
        df['cloud_coverage'] = df['cloud_coverage']
        del median
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [17]:
class CloudTimeCat(TransformerMixin):
        
    def transform(self, df, **transform_params):
        tempDf = df[['cloud_coverage', 'hour']].astype('int')
        tempDf['cloud_coverage'] = (tempDf['cloud_coverage']).astype('int')
        tempDf['hour'] = (tempDf['hour']).astype('int')
        tempDf = tempDf.astype('str')
        df['cloud_time_cat'] = 'c' + tempDf['cloud_coverage'] + 't' + tempDf['hour']
        df['cloud_time_cat'] = df['cloud_time_cat'].astype('category')
        del tempDf
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [18]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [19]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        year_built_median = df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        del year_built_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [20]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [21]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('category') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = (df['timestamp'].dt.dayofweek * 24) + df['timestamp'].dt.hour
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('category')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [22]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [23]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [24]:
class FillZeros(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(0)
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [25]:
class FillMedian(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].median())
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [26]:
class FillPopular(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].value_counts()[0])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [27]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
class GC(TransformerMixin):
        
    def transform(self, df, **transform_params):
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [29]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)

def lbm_rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

# rob's custome function to do RMSLE while in the log1p space
def lbm_rmslee(y_true, y_pred):
    return 'RMSLEE', np.sqrt(np.mean(np.power(y_pred - y_true, 2))), False



In [30]:
weather_pipes = Pipeline(
    steps=[
        #('convertToDatetime', ConvertToDatetime()),
        ('imputeWeather', ImputeWeather()),
        ('fillMean',FillMean(['air_temperature','dew_temperature'
                              , 'precip_depth_1_hr', 'sea_level_pressure'])),
        ('imputeCloudCoverage', ImputeCloudCoverage()),
        ('addWeatherLags3', AddWeatherLags(3)),
        ('addWeatherLags72', AddWeatherLags(72)),
    ]
)

building_pipes = Pipeline(
    steps=[
        ('logSquareFeet', LogSquareFeet()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('fillMean',FillMean(['floor_count'])),
        ('dropClos', DropCols(['square_feet', 'year_built'])),
    ]
)


# pre_a_pipes is for preprocessing that doesn't change impute
# values
x_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        #('convertToDatetime', ConvertToDatetime()),
        ('rmS0M0', RmS0M0()),
        #('addHolidays', AddHolidays()),
        #('rmHolidays', RmHolidays()), called manually in fold
        ('addRelativeHumidity',AddRelativeHumidity()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('setCatTypes', SetCatTypes()),
        ('fillMean',FillMean([])),
        ('fillZeros',FillZeros([])),
        ('dropClos', DropCols(['timestamp'])),
        ('GC', GC())
    ]
)

In [38]:
def getOutsideFoldXY(train_index):
    X = train.iloc[train_index].drop('meter_reading', axis=1)
    X_buildings = building[building['building_id'].isin(X['building_id'].unique())]
    X_weather = building[building['building_id'].isin(X['building_id'].unique())]
    X = x_pipes.transform(
        X
            .merge(building_pipes.transform(X_buildings), on='building_id', how='left')
            .merge(weather_train_trans, on=['site_id', 'timestamp'], how='left')
        )
    f_train_y = np.log1p(train.iloc[train_index]['meter_reading'])
    print(X.columns)
    return X,f_train_y



def getInFoldXY(train_index):
    X = train.iloc[train_index]
    X_buildings = building[building['building_id'].isin(X['building_id'].unique())]
    X = X.merge(building_pipes.transform(X_buildings), on='building_id', how='left')
    X_weather = weather_train[
        (weather_train['site_id'].isin(X['site_id'].unique())) 
         & (weather_train['timestamp'].isin(X['timestamp'].unique())) 
    ]
    X = x_pipes.transform(
        rmHolidays.transform(
            X.merge(weather_pipes.transform(X_weather), how='left')))
    return X.drop('meter_reading', axis=1),  np.log1p(X['meter_reading'])

sample_train_X, sample_train_y = getInFoldXY(train.sample(n=20,  random_state=42).index)
print(sample_train_X)

   building_id meter site_id                    primary_use  floor_count  \
0         1324     1      14  Entertainment/public assembly     2.800781   
1         1013     0      10                      Education     3.000000   
2          229     1       2                      Education     2.800781   
3          217     3       2                      Education     2.800781   
4         1434     0      15                      Education     2.800781   
5         1047     0      12                Public services     2.800781   
6          911     1       9                      Education     2.800781   
7         1039     0      12                      Education     3.000000   
8          265     0       2                         Office     2.800781   
9          896     0       9                      Education     2.800781   
10         973     0       9                         Office     2.800781   
11         813     0       8                Public services     1.000000   
12         8

In [32]:
# model fit params
gbm_params = {
    'n_estimators' : 500, # for accuracy use large numbers like 6000 
    'learning_rate': 0.9,
    'feature_fraction' : 0.9,
    'subsample' : 0.1,  # 
    'subsample_freq' : 1,
    'num_leaves' : 20,
    'max_depth' : 10,
    'metric':'rmse',
    'lambda_l1' : 1,  
    'lambda_l2': 1,
    'verbose': 100
}

In [33]:
%%time
## cross val mdoels

folds = 5

# this stratified strategy from
# https://www.kaggle.com/isaienkov/lightgbm-fe-1-19/notebook
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

models = []
best_scores = []
for train_index, val_index in kf.split(train, train['building_id']):
    f_train_X, f_train_y = getInFoldXY(train_index)
    f_val_X, f_val_y = getInFoldXY(val_index)
    gbm = LGBMRegressor(**gbm_params)
    gbm.fit(f_train_X, f_train_y,
        eval_set=[(f_val_X, f_val_y)],
        # https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114722#latest-660848
        # eval_metric=lbm_rmslee,
        early_stopping_rounds=20)
    models.append(gbm)
    #y_pred = gbm.predict(f_val_X, num_iteration=gbm.best_iteration_)
    # eval
    #rmsle_score = lbm_rmslee(f_val_X, y_pred)[1]
    best_scores.append(gbm.best_score_)
    del f_train_X, f_train_y, f_val_X, f_val_y, gbm
    gc.collect()


[1]	valid_0's rmse: 1.59507
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.32164
[3]	valid_0's rmse: 1.27539
[4]	valid_0's rmse: 1.25253
[5]	valid_0's rmse: 1.19604
[6]	valid_0's rmse: 1.18696
[7]	valid_0's rmse: 1.16634
[8]	valid_0's rmse: 1.15374
[9]	valid_0's rmse: 1.14526
[10]	valid_0's rmse: 1.10838
[11]	valid_0's rmse: 1.0804
[12]	valid_0's rmse: 1.07305
[13]	valid_0's rmse: 1.06791
[14]	valid_0's rmse: 1.06442
[15]	valid_0's rmse: 1.06064
[16]	valid_0's rmse: 1.05604
[17]	valid_0's rmse: 1.05131
[18]	valid_0's rmse: 1.03383
[19]	valid_0's rmse: 1.03035
[20]	valid_0's rmse: 1.02762
[21]	valid_0's rmse: 1.02521
[22]	valid_0's rmse: 1.02383
[23]	valid_0's rmse: 1.02126
[24]	valid_0's rmse: 1.01844
[25]	valid_0's rmse: 1.0163
[26]	valid_0's rmse: 1.01286
[27]	valid_0's rmse: 1.01053
[28]	valid_0's rmse: 1.0052
[29]	valid_0's rmse: 1.00319
[30]	valid_0's rmse: 1.00159
[31]	valid_0's rmse: 0.996441
[32]	valid_0's rmse: 0.994494
[33]	valid_0's rmse: 

[268]	valid_0's rmse: 0.853322
[269]	valid_0's rmse: 0.853058
[270]	valid_0's rmse: 0.852934
[271]	valid_0's rmse: 0.853123
[272]	valid_0's rmse: 0.853454
[273]	valid_0's rmse: 0.853835
[274]	valid_0's rmse: 0.854141
[275]	valid_0's rmse: 0.853765
[276]	valid_0's rmse: 0.854063
[277]	valid_0's rmse: 0.8541
[278]	valid_0's rmse: 0.853972
[279]	valid_0's rmse: 0.85381
[280]	valid_0's rmse: 0.853586
[281]	valid_0's rmse: 0.853279
[282]	valid_0's rmse: 0.85323
[283]	valid_0's rmse: 0.85315
[284]	valid_0's rmse: 0.853279
[285]	valid_0's rmse: 0.853613
[286]	valid_0's rmse: 0.853568
[287]	valid_0's rmse: 0.853504
[288]	valid_0's rmse: 0.853919
[289]	valid_0's rmse: 0.853803
[290]	valid_0's rmse: 0.853676
Early stopping, best iteration is:
[270]	valid_0's rmse: 0.852934
[1]	valid_0's rmse: 1.61095
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.32882
[3]	valid_0's rmse: 1.28255
[4]	valid_0's rmse: 1.25717
[5]	valid_0's rmse: 1.23679
[6]	valid_0's rmse: 1.157

[243]	valid_0's rmse: 0.857521
[244]	valid_0's rmse: 0.857411
[245]	valid_0's rmse: 0.856908
[246]	valid_0's rmse: 0.856527
[247]	valid_0's rmse: 0.856157
[248]	valid_0's rmse: 0.855989
[249]	valid_0's rmse: 0.856092
[250]	valid_0's rmse: 0.855906
[251]	valid_0's rmse: 0.855787
[252]	valid_0's rmse: 0.855769
[253]	valid_0's rmse: 0.856088
[254]	valid_0's rmse: 0.855946
[255]	valid_0's rmse: 0.85593
[256]	valid_0's rmse: 0.855593
[257]	valid_0's rmse: 0.855573
[258]	valid_0's rmse: 0.855368
[259]	valid_0's rmse: 0.855198
[260]	valid_0's rmse: 0.855033
[261]	valid_0's rmse: 0.854715
[262]	valid_0's rmse: 0.854559
[263]	valid_0's rmse: 0.8543
[264]	valid_0's rmse: 0.85457
[265]	valid_0's rmse: 0.85475
[266]	valid_0's rmse: 0.854515
[267]	valid_0's rmse: 0.854778
[268]	valid_0's rmse: 0.854063
[269]	valid_0's rmse: 0.854051
[270]	valid_0's rmse: 0.853952
[271]	valid_0's rmse: 0.853835
[272]	valid_0's rmse: 0.853651
[273]	valid_0's rmse: 0.853503
[274]	valid_0's rmse: 0.853486
[275]	valid_0

[212]	valid_0's rmse: 0.864431
[213]	valid_0's rmse: 0.864366
[214]	valid_0's rmse: 0.864156
[215]	valid_0's rmse: 0.864062
[216]	valid_0's rmse: 0.863941
[217]	valid_0's rmse: 0.863897
[218]	valid_0's rmse: 0.863432
[219]	valid_0's rmse: 0.863213
[220]	valid_0's rmse: 0.862904
[221]	valid_0's rmse: 0.862539
[222]	valid_0's rmse: 0.862402
[223]	valid_0's rmse: 0.862145
[224]	valid_0's rmse: 0.862146
[225]	valid_0's rmse: 0.862283
[226]	valid_0's rmse: 0.862234
[227]	valid_0's rmse: 0.861748
[228]	valid_0's rmse: 0.861461
[229]	valid_0's rmse: 0.861171
[230]	valid_0's rmse: 0.86128
[231]	valid_0's rmse: 0.861356
[232]	valid_0's rmse: 0.860488
[233]	valid_0's rmse: 0.86031
[234]	valid_0's rmse: 0.860081
[235]	valid_0's rmse: 0.860026
[236]	valid_0's rmse: 0.859925
[237]	valid_0's rmse: 0.859869
[238]	valid_0's rmse: 0.859892
[239]	valid_0's rmse: 0.859899
[240]	valid_0's rmse: 0.859704
[241]	valid_0's rmse: 0.859714
[242]	valid_0's rmse: 0.859544
[243]	valid_0's rmse: 0.859431
[244]	vali

[168]	valid_0's rmse: 0.867402
[169]	valid_0's rmse: 0.867346
[170]	valid_0's rmse: 0.86725
[171]	valid_0's rmse: 0.867192
[172]	valid_0's rmse: 0.866795
[173]	valid_0's rmse: 0.866897
[174]	valid_0's rmse: 0.866715
[175]	valid_0's rmse: 0.86674
[176]	valid_0's rmse: 0.867002
[177]	valid_0's rmse: 0.866979
[178]	valid_0's rmse: 0.866923
[179]	valid_0's rmse: 0.866714
[180]	valid_0's rmse: 0.86671
[181]	valid_0's rmse: 0.866571
[182]	valid_0's rmse: 0.866593
[183]	valid_0's rmse: 0.866831
[184]	valid_0's rmse: 0.866635
[185]	valid_0's rmse: 0.866325
[186]	valid_0's rmse: 0.865992
[187]	valid_0's rmse: 0.865981
[188]	valid_0's rmse: 0.866242
[189]	valid_0's rmse: 0.866046
[190]	valid_0's rmse: 0.865781
[191]	valid_0's rmse: 0.865133
[192]	valid_0's rmse: 0.865034
[193]	valid_0's rmse: 0.864872
[194]	valid_0's rmse: 0.86434
[195]	valid_0's rmse: 0.864251
[196]	valid_0's rmse: 0.863971
[197]	valid_0's rmse: 0.863867
[198]	valid_0's rmse: 0.863762
[199]	valid_0's rmse: 0.863559
[200]	valid_

[177]	valid_0's rmse: 0.859674
[178]	valid_0's rmse: 0.859673
[179]	valid_0's rmse: 0.85973
[180]	valid_0's rmse: 0.859793
[181]	valid_0's rmse: 0.859691
[182]	valid_0's rmse: 0.859712
[183]	valid_0's rmse: 0.859927
[184]	valid_0's rmse: 0.859951
[185]	valid_0's rmse: 0.859745
[186]	valid_0's rmse: 0.859704
[187]	valid_0's rmse: 0.859453
[188]	valid_0's rmse: 0.859161
[189]	valid_0's rmse: 0.858995
[190]	valid_0's rmse: 0.858741
[191]	valid_0's rmse: 0.858587
[192]	valid_0's rmse: 0.858508
[193]	valid_0's rmse: 0.858456
[194]	valid_0's rmse: 0.858161
[195]	valid_0's rmse: 0.857714
[196]	valid_0's rmse: 0.857551
[197]	valid_0's rmse: 0.857484
[198]	valid_0's rmse: 0.857262
[199]	valid_0's rmse: 0.856936
[200]	valid_0's rmse: 0.856754
[201]	valid_0's rmse: 0.856844
[202]	valid_0's rmse: 0.856758
[203]	valid_0's rmse: 0.856708
[204]	valid_0's rmse: 0.856621
[205]	valid_0's rmse: 0.856319
[206]	valid_0's rmse: 0.856258
[207]	valid_0's rmse: 0.856278
[208]	valid_0's rmse: 0.85596
[209]	vali

In [34]:
# Scores for cross val
for score in best_scores:
    print(score['valid_0']['rmse'])

0.8529335864492671
0.852878522512121
0.8561534572606718
0.8565740757613876
0.852763554623322


In [39]:
# Importance rank for first model in cross val models
imprtc_df = pd.DataFrame()
imprtc_df['feature'] = sample_train_X.columns   
imprtc_df['importance'] = models[0].feature_importances_
imprtc_df.sort_values('importance', ascending=False, inplace= True)
print(imprtc_df)
print(sample_train_X.columns)

                          feature  importance
0                     building_id        2198
1                           meter         468
71                           hour         381
42     air_temperature_mean_lag72         136
29     precip_depth_1_hr_std_lag3         122
44      air_temperature_min_lag72         111
43      air_temperature_max_lag72         108
5                 log_square_feet         102
14      air_temperature_mean_lag3          93
52      dew_temperature_min_lag72          79
50     dew_temperature_mean_lag72          73
59   sea_level_pressure_max_lag72          71
51      dew_temperature_max_lag72          65
62      wind_direction_mean_lag72          55
2                         site_id          53
60   sea_level_pressure_min_lag72          51
66          wind_speed_mean_lag72          44
46      cloud_coverage_mean_lag72          44
57    precip_depth_1_hr_std_lag72          43
16       air_temperature_min_lag3          43
45      air_temperature_std_lag72 

In [40]:
%%time
## Single fit single model

gbm = LGBMRegressor(**gbm_params)
f_train_X, f_train_y = getInFoldXY(train.index)
gbm.fit(f_train_X, f_train_y)

Wall time: 4min 51s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.9, importance_type='split', lambda_l1=1,
              lambda_l2=1, learning_rate=0.9, max_depth=10, metric='rmse',
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=500, n_jobs=-1, num_leaves=20, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=0.1, subsample_for_bin=200000, subsample_freq=1,
              verbose=100)

In [41]:
# Generate test_X
test_X = x_pipes.transform(
    test
        .merge(building_pipes.transform(building), on='building_id', how='left').drop(['row_id'], axis=1)
        .merge(weather_pipes.transform(weather_test), on=['site_id', 'timestamp'], how='left')
    )

print(test_X.sample(n=20,  random_state=42))
print(test_X.shape)

         building_id meter site_id                    primary_use  \
3573457          173     0       2                      Education   
8315486          222     1       2  Entertainment/public assembly   
40305643        1354     2      15                      Education   
16083617         712     0       5                      Education   
37204119        1344     2      15                      Education   
32144852        1119     1      13                         Office   
5105044          249     0       2  Entertainment/public assembly   
36982844        1303     1      14                     Healthcare   
20487823         945     2       9                         Office   
8404196          217     1       2                      Education   
6889602          241     0       2  Entertainment/public assembly   
16963616         784     0       6                      Education   
39666699        1381     2      15                         Office   
26802058        1179     1      13

[20 rows x 72 columns]
(41697600, 72)


In [ ]:
# Predict using cross val models ensemble 
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
    res.append(np.expm1(sum([model.predict(test_X.iloc[i:i+step_size]) for model in models])/folds))
    #res.append(np.expm1(gbm.predict(test_X.iloc[i:i+step_size])))
    i+=step_size
    
    
    


 79%|██████████████████████████████████████████████████████████████▊                 | 655/834 [20:02<05:38,  1.89s/it]

In [ ]:
# Save using cross val models ensemble 
res = np.concatenate(res)
print(len(res))
submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission.csv.zip', index=False)
submission.shape

In [ ]:
# Predict single model fit
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
    #res.append(np.expm1(sum([model.predict(test_X.iloc[i:i+step_size]) for model in models])/folds))
    res.append(np.expm1(gbm.predict(test_X.iloc[i:i+step_size])))
    i+=step_size
    

In [ ]:
# Save single model fit
res = np.concatenate(res)
print(len(res))
submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_single.csv.zip', index=False)
submission.shape